In [8]:
import logging

import pandas as pd
import tqdm
import tsplib95
from joblib import Parallel, delayed

from ag import GA
from domain.crossovers import VRCrossOver, PMXCrossOver, APCrossOver, CXCrossOver, OX1CrossOver, OX2CrossOver, \
    POSCrossOver, ERCrossOver
from domain.mutations import DM, EM, ISM, SIM, SM, IVM
from domain.path.euclidean_extractor import EuclideanPathParentExtractor
from domain.path.path_population import PathPopulation

In [9]:
def run_al(cross_over, mutation, problem):
    try:
        logging.warning(f" Running: {cross_over.__class__.__name__} x {mutation.__class__.__name__}")
        ag = GA()
        ag.set_population_class(PathPopulation())
        ag.set_parent_extractor_class(EuclideanPathParentExtractor())
        ag.set_crossover_class(cross_over)
        ag.set_mutation_class(mutation)
        distance, path = ag.perform(problem, 200)
        print(f" {cross_over.__class__.__name__} x {mutation.__class__.__name__}: Best Path[{distance}]: {path}")
        return cross_over.__class__.__name__, mutation.__class__.__name__,distance, path
    except Exception as e:
        logging.warning(f" Couldn't do the comb: {cross_over} x {mutation} : {e}")
        return cross_over.__class__.__name__, mutation.__class__.__name__,None, None

In [10]:
cross_overs = [APCrossOver(), CXCrossOver(), ERCrossOver(), OX1CrossOver(), OX2CrossOver(), PMXCrossOver(), POSCrossOver(), VRCrossOver()]

In [11]:
mutations = [DM(), EM(), ISM(), IVM(), SIM(), SM()]

In [12]:
combinations = tqdm.tqdm([(co, mt) for mt in mutations for co in cross_overs])


  0%|          | 0/48 [00:00<?, ?it/s]

In [13]:
problem = tsplib95.load('../data/gr24.tsp')
best_solution = tsplib95.load("../data/gr24.opt.tour")

In [14]:
results = Parallel(n_jobs=len(mutations))(delayed(run_al)(comb[0], comb[1], problem) for comb in combinations)


100%|██████████| 48/48 [12:00<00:00, 15.02s/it]


In [15]:
collected = []
for (co, mt, dist, path) in results:
    value = list(filter(lambda a: a["mutation"] == mt, collected))
    if len(value) == 0:
        value = {"mutation": mt}
        collected.append(value)
    else:
        value = value[0]
    value[co] = dist


In [16]:
df = pd.DataFrame(collected)
df.to_csv("problem24.csv")

In [17]:
problem = tsplib95.load('../data/gr48.tsp')
best_solution = tsplib95.load("../data/gr48.opt.tour")

In [18]:
results = Parallel(n_jobs=len(mutations))(delayed(run_al)(comb[0], comb[1], problem) for comb in combinations)

In [19]:
collected = []
for (co, mt, dist, path) in results:
    value = list(filter(lambda a: a["mutation"] == mt, collected))
    if len(value) == 0:
        value = {"mutation": mt}
        collected.append(value)
    else:
        value = value[0]
    value[co] = dist


In [20]:
df = pd.DataFrame(collected)
df.to_csv("problem48.csv")